# A Common Hierarchical Abstract Representation of Music: applications in historical musicology

Given a more-or-less comprehensive corpus of encoded historical lute tablatures like the augmented ‘Electronic Corpus of Lute Music’ (ECOLM) currently under development, an entirely new mode of investigation of the corpus and its relation to other repertories, such as contemporary vocal or keyboard music, could become possible. Text-based encodings of works may be directly compared using exact matching, but this does not allow recognition of transposed versions, or those for a differently-tuned instrument than a query; furthermore, duple- and triple-time versions of the same music will not, in general, be matched, nor those in which durations have been globally augmented or diminished. While ingenious indexing strategies might overcome some of these problems for a particular repertory of music (for example, the 16th-century lute fantasy and related genres), they are unlikely to permit extra-corpus searches for parallel motives or passages within contemporary vocal music, which have long been recognised as crucial in the evolution of western lute music.  
  
We present CHARM, an abstract, hierarchical music knowledge representation system that admits federation of data sources in multiple formats by means of semantic-level annotation and query specification. From the user’s perspective, search and discovery operations are specified in terms of musical terminology applied to “constituents” (groups of musical objects – e.g., notes, phrases, motives, etc.), independently of the data source and format. CHARM is equipped with data interfaces for MIDI and TabCode (other formats in progress), and it can represent audio recordings, allowing detailed association of individual notes with specific spectral content, permitting the annotation of richly detailed analyses of sources and their direct, detailed connection with performance recordings.

In this notebook we demonstrate how CHARM can be used to construct music knowledge bases from encodings of lute tablature. Using CHARM to construct knowledge bases requires the implementation of interfaces for concrete data sources and engineering the coordination of these interfaces. Here we use the Julia programming language, which has an expressive type system and method overloading, the combination of which is exploited to implement interfaces with type-based dynamic dispatch.

## CHARM

In [3]:
module CHARM

export option, none, fnd, geta, getp, pts, dom, cts

struct None end
none = None()
option{A} = Union{A,None}

# Abstract Types

abstract type Id end
abstract type Constituent end
abstract type Hierarchy end
abstract type Attribute{N,T} end
abstract type Property{N,T} end

# Registered Attributes and Properties 

__attributes__(::Val{N}) where N = error("No attribute named $N.")
__attributes__(N::Symbol) = __attributes__(Val{N}())
__properties__(::Val{N}) where N = error("No property named $N.")
__properties__(N::Symbol) = __properties__(Val{N}())

# Interface Operations

fnd(x::Id,h::Hierarchy) = none
fnd(x::Id,m::Module) = fnd(x,m.__data__)

geta(::Attribute,c::Constituent) = none
geta(N::Symbol,c) = geta(__attributes__(Val{N}()),c)

getp(::Property,c::Constituent) = none
getp(N::Symbol,c) = getp(__properties__(Val{N}()),c)

pts(c::Constituent) = Id[]

dom(h::Hierarchy) = Id[]
dom(m::Module) = dom(m.__data__)

#sequence(ids,kb) = [fnd(x,kb) for x in ids]
#cts(kb) = sequence(dom(kb),kb)

cts(kb) = Pair{Id,Constituent}[x=>fnd(x,kb) for x in dom(kb)]

sequence(ids,kb) = begin
    isempty(ids) && return Constituent[]
    c = fnd(ids[1],kb)
    c isa None && return none
    r = sequence(ids[2:end],kb)
    r isa None && return none
    return Constituent[c,r...]
end

# PROPERTIES

export DESCRIPTION, TYPE

struct DESCRIPTION <: Property{:DESCRIPTION,String} end
__properties__(::Val{:DESCRIPTION}) = DESCRIPTION()

struct TYPE <: Property{:TYPE,String} end
__properties__(::Val{:TYPE}) = TYPE()

# ATTRIBUTES

export PITCH, DUR

struct PITCH <: Attribute{:PITCH,Int} end
__attributes__(::Val{:PITCH}) = PITCH()

struct DUR <: Attribute{:DUR,String} end
__attributes__(::Val{:DUR}) = DUR()

end

using Main.CHARM

### CHARM Query Language

In [5]:
PAIR(X,Y) = Pair{x,y} where {x<:X,y<:Y}
LST(X) = Vector{x} where {x<:X}
OPT(X) = Union{X,CHARM.None}
ID = CHARM.Id
C = CHARM.Constituent
CON = PAIR(ID,C)
G = LST(CON)

domain(cs::G) = first.(cs)

mem(x::ID,cs::G)::Bool = x in domain(cs)
mem(xs::LST(ID),cs::G) = all([mem(x,cs) for x in xs])

position(x::ID,cs::G) = findfirst(==(x),domain(cs))

lookup(x::ID,cs::G) = mem(x,cs) ? cs[position(x,cs)] : none

lookup(xs::LST(ID),cs::G)::option{CON} = begin
    res = Pair{CHARM.Id,CHARM.Constituent}[]
    for x in xs
        c = lookup(x,cs)
        c == none && return none
        union!(res,[c])
    end
    return res
end

pop(x::ID,cs::G) = begin 
    !mem(x,cs) && return CON[]
    return cs[position(x,cs):end]
end

parts(x::ID,cs::G) = begin 
    !mem(x,cs) && return none
    c = lookup(x,cs)
    return lookup(pts(c[2]),cs)
end
parts(c::CON,cs::G) = lookup(pts(c[2]),cs)

subparts(x::ID,cs::G) = begin
    isempty(cs) && return CON[]
    c = lookup(x,cs)
    c == none && return CON[]
    res = CON[c]
    ps = pts(c[2])
    for p in ps
        union!(res,subparts(p,pop(p,cs)))
    end
    return res
end

subparts(xs::LST(ID),cs::G) = begin
    res = CON[]
    [union!(res,subparts(x,cs)) for x in xs]
    return [c for c in cs if c in res]
end

superparts(x::ID,cs::G) = begin
    isempty(cs) && return CON[]
    c = lookup(x,cs)
    c == none && return CON[]
    res = CON[c]
    for c2 in reverse(cs[1:position(x,cs)])
        if x in pts(c2[2])
            push!(res,c2)
            union!(res,superparts(c2[1],cs))
        end
    end
    return reverse(res)
end

superparts(xs::LST(ID),cs::G) = begin 
    res = CON[]
    [union!(res,superparts(x,cs)) for x in xs]
    return [c for c in cs if c in res]
end

haspart(x::ID,y::ID,cs::G) = begin
    !(mem(x,cs)) && return false
    return y in first.(parts(x,cs))
end

ispartof(x::ID,y::ID,cs::G) = haspart(y,x,cs)

hassubpart(x,y,cs) = begin
    y in first.(subparts(x,cs))
end

issubpartof(x::ID,y::ID,cs::G) = hassubpart(y,x,cs)

isdag(cs::G) = begin 
    isempty(cs) && return true
    x,c = cs[1]
    t = cs[2:end]
    ps = pts(c)
    mem(x,t) && (print("!!!"); return false)
    !mem(ps,t) && (print("!!!$x"); return false)
    !isdag(t) && (print("!!!"); return false)
    return true
end
isleaf(x::ID,cs::G) = begin 
    c = lookup(x,cs) 
    return c != none && isempty(pts(c[2]))
end
istree(cs::G) = begin
    isempty(cs) && return true
    x,c = cs[1]
    t = cs[2:end]
    sps = [subparts(p[1],cs) for p in parts(x,cs)]
    for p in parts(x)
        return isleaf(p,cs) || istree(p,cs)
    end
end

PTY{N,T} = CHARM.Property{N,t} where {t<:T}
ATT{N,T} = CHARM.Attribute{N,t} where {t<:T}

# READ(A) : G -> option A x G 

abstract type READ{A} end

(R::READ{A})(cs::G) where A = R.op(cs)::option{A}

struct Read{A} <: READ{A}
    op::Function
    Read(A,f) = new{A}(f)
    Read(A) = new{A}(cs->none)
end

abstract type KLEISLI{A,B} end

function (K::KLEISLI{A,B})(a) where {A,B}
    a==none && return Read(B)
    Read(B,K.op(a))
end

function(K::KLEISLI{A,B})(a::A,cs::G) where {A,B}
    K(a)(cs)
end

struct BIND{A,B} <: READ{B}
    op
    BIND(R::READ{A},F::KLEISLI{A,B}) where {A,B} = new{A,B}(cs->F(R(cs),cs))
end

struct COMP{A,B} <: KLEISLI{A,B}
    op
    COMP(F::KLEISLI{A,B},G::KLEISLI{B,C}) where {A,B,C} = new{A,C}(x->cs->G(F(x,cs),cs))
end

COMP(f::KLEISLI,gs::KLEISLI...) = isempty(gs) ? f : COMP(f,COMP(gs...))

QUERY(R::READ,KS::KLEISLI...) = BIND(R,COMP(KS...))

struct PTS <: KLEISLI{CON,LST(CON)}
    op
    PTS() = new(c->cs->parts(c,cs))
end

struct GETA{N,T} <: KLEISLI{CON,T}
    op
    GETA(a::ATT{N,T}) where {N,T} = new{N,T}(c->cs->geta(a,c[2]))
    GETA(N::Symbol) = GETA(CHARM.__attributes__(N))
end

struct GETP{N,T} <: KLEISLI{CON,T}
    op
    GETP(a::PTY{N,T}) where {N,T} = new{N,T}(c->cs->getp(a,c[2]))
    GETP(N::Symbol) = GETP(CHARM.__properties__(N))
end

struct HASA <: KLEISLI{CON,Bool}
    op
    HASA(N::Symbol,v) = new(c->cs->geta(N,c[2])==v)
    HASA(N::Symbol) = new(c->cs->geta(N,c[2])!=none)
end

struct HASP <: KLEISLI{CON,Bool}
    op
    HASP(N::Symbol,v) = new(c->cs->getp(N,c[2])==v)
    HASP(N::Symbol) = new(c->cs->getp(N,c[2])!=none)
end

struct FIRSTPART <: KLEISLI{CON,Bool}
    op
    FIRSTPART(T::KLEISLI) = new(c->cs->(ps = parts(c,cs); isempty(ps) ? false : T(first(ps))(cs)))
    FIRSTPART(x::ID) = FIRSTPART(c->cs->c[1]==x)
end

struct LASTPART <: KLEISLI{CON,Bool}
    op
    LASTPART(T) = new(c->cs->(ps = parts(c,cs); isempty(ps) ? false : T(last(ps))(cs)))
    LASTPART(x::ID) = LASTPART(c->cs->c[1]==x)
end

struct ALLPARTS <: KLEISLI{CON,Bool}
    op
    ALLPARTS(T) = new(c->cs->all([T(p,cs) for p in parts(c,cs)])) 
end

struct SOMEPART <: KLEISLI{CON,Bool}
    op
    SOMEPART(T) = new(c->cs->any([T(p,cs) for p in parts(c,cs)]))
end

struct AND <: KLEISLI{CON,Bool}
    op
    AND(f::KLEISLI,g::KLEISLI) = new(a->cs->f(a,cs)&&g(a,cs))
    AND(f::KLEISLI,gs::KLEISLI...) = isempty(gs) ? f : AND(AND(f,gs[1]),gs[2:end]...)
end

struct SELECT <: READ{G}
    op
    SELECT(T::KLEISLI{CON,Bool}) = new(cs->[c for c in cs if T(c)(cs)])
    SELECT(x) = new(cs->[lookup(x,cs)])
end

opmap(B,f,l) = begin
    isempty(l) && return B[]
    h = f(l[1])
    t = opmap(B,f,l[2:end])
    h == none && return none
    t == none && return none
    return B[h,t...]
end

struct MAP{A,B} <: KLEISLI{LST(A),LST(B)}
    op
    MAP(f::KLEISLI{A,B}) where {A,B} = new{A,B}(l->cs->opmap(B,x->f(x)(cs),l))
end

struct RET{A} <: READ{A}
    op
    RET(v::A) where A = new{A}(_->v)
end

struct SUBCS <: KLEISLI{CON,G}
    op
    SUBCS() = new(c->cs->subparts(c,cs))
end

struct PROJ <: KLEISLI{CON,G}
    op
    PROJ(T::KLEISLI{CON,Bool}) = new(c->cs->CON[x for x in subparts(c[1],cs) if T(x,cs)])
end

struct SUM <: KLEISLI{LST(Float64),Float64}
    op
    SUM() = new(l->cs->sum(l))
end

struct AVG <: KLEISLI{LST(Float64),Float64}
    op
    AVG() = new(l->cs->sum(l)/length(l))
end

## Tabcode Example

```
{<rules>
<tuning_named>renaissance</tuning_named>
<pitch>67</pitch>
 <rhythm-font>varietie</rhythm-font>
<bass_tuning>(-2)</bass_tuning>
</rules>}
M(C/)
Qa1a2b3c4a6
E.c4
Td2
b2
Ea2b3c4
d3
Sd1a2
b2
a2
d3
|
Sa1b3d5
d3
b3
a3
c4d5
a3
b3
d3
e2a3c5
c2
a2
c2
e2a3c5
a1
c1
e2
|
```

## Parse to JSON with Parsetab

Install parsetab: https://github.com/jamieforth/parsetab

Convert tabcode `input.tc` to json.

```
parsetab tc2json input.tc
```

```json
[{"code":"{<rules>\n<tuning_named>renaissance</tuning_named>\n<pitch>67</pitch>\n <rhythm-font>varietie</rhythm-font>\n<bass_tuning>(-2)</bass_tuning>\n</rules>}","fType":"Ruleset","rules":{"notation":"French","pitch":67,"tuning_named":"renaissance","rhythm-font":"varietie","bass_tuning":"(-2)"}},{"code":"M(C/)","tType":"Metre","components":["C/"],"vertical":false},{"tType":"Chord","duration":{"code":"Q","tType":"Duration"},"mainCourses":[{"tType":"TabNote","pitch":{"code":"a1","tType":"Pitch","fret":"a","course":1}},{"tType":"TabNote","pitch":{"code":"a2","tType":"Pitch","fret":"a","course":2}},{"tType":"TabNote","pitch":{"code":"b3","tType":"Pitch","fret":"b","course":3}},{"tType":"TabNote","pitch":{"code":"c4","tType":"Pitch","fret":"c","course":4}},{"tType":"TabNote","pitch":{"code":"a6","tType":"Pitch","fret":"a","course":6}}],"bassCourses":[]},{"tType":"Chord","duration":{"code":"E.","tType":"Duration"},"mainCourses":[{"tType":"TabNote","pitch":{"code":"c4","tType":"Pitch","fret":"c","course":4}}],"bassCourses":[]},{"tType":"Chord","duration":{"code":"T","tType":"Duration"},"mainCourses":[{"tType":"TabNote","pitch":{"code":"d2","tType":"Pitch","fret":"d","course":2}}],"bassCourses":[]},{"tType":"Chord","duration":{"code":"T","tType":"Duration"},"mainCourses":[{"tType":"TabNote","pitch":{"code":"b2","tType":"Pitch","fret":"b","course":2}}],"bassCourses":[]},{"tType":"Chord","duration":{"code":"E","tType":"Duration"},"mainCourses":[{"tType":"TabNote","pitch":{"code":"a2","tType":"Pitch","fret":"a","course":2}},{"tType":"TabNote","pitch":{"code":"b3","tType":"Pitch","fret":"b","course":3}},{"tType":"TabNote","pitch":{"code":"c4","tType":"Pitch","fret":"c","course":4}}],"bassCourses":[]},{"tType":"Chord","duration":{"code":"E","tType":"Duration"},"mainCourses":[{"tType":"TabNote","pitch":{"code":"d3","tType":"Pitch","fret":"d","course":3}}],"bassCourses":[]},{"tType":"Chord","duration":{"code":"S","tType":"Duration"},"mainCourses":[{"tType":"TabNote","pitch":{"code":"d1","tType":"Pitch","fret":"d","course":1}},{"tType":"TabNote","pitch":{"code":"a2","tType":"Pitch","fret":"a","course":2}}],"bassCourses":[]},{"tType":"Chord","duration":{"code":"S","tType":"Duration"},"mainCourses":[{"tType":"TabNote","pitch":{"code":"b2","tType":"Pitch","fret":"b","course":2}}],"bassCourses":[]},{"tType":"Chord","duration":{"code":"S","tType":"Duration"},"mainCourses":[{"tType":"TabNote","pitch":{"code":"a2","tType":"Pitch","fret":"a","course":2}}],"bassCourses":[]},{"tType":"Chord","duration":{"code":"S","tType":"Duration"},"mainCourses":[{"tType":"TabNote","pitch":{"code":"d3","tType":"Pitch","fret":"d","course":3}}],"bassCourses":[]},
```

## CHARM Interface for Tabcode

In [12]:
module Tabcode

using Main.CHARM, JSON

function parse(fn)

    hasttype(s,g) = haskey(g,"tType") && g["tType"] == s

    gs = JSON.parsefile(fn)
    
    notes, chords, bars, systems, pages = [], [], [], [], []
    n, c, b, s, p = 1, 1, 1, 1, 1
    ns, cs, bs, ss, ps = [], [], [], [], []
    
    for g in gs
        if hasttype("Chord",g)
            push!(cs,c)        
            
            dur = g["duration"]["code"]
    
            for x in g["mainCourses"]
                push!(ns,n)
                
                course = x["pitch"]["course"]
                fret = x["pitch"]["fret"]
                
                note = (id=n,course=course,fret=fret,dur=dur,c=c,b=b,s=s,p=p)
                push!(notes,note)
                n+=1
            end
    
            chord = (id=c,dur=dur,b=b,s=s,p=p,notes=ns)
            push!(chords,chord)
            
            ns = []
            c+=1
        end
        if hasttype("Barline",g)
            push!(bs,b)
            bar = (id=b,s=s,p=p,chords=cs)
            push!(bars,bar)
            cs = []
            b+=1
        end
        if hasttype("SystemBreak",g)
            push!(ss,s)
            system = (id=s,p=p,bars=bs)
            push!(systems,system)
            bs = []
            s+=1
        end
        if hasttype("PageBreak",g)
            push!(ps,p)
            page = (id=p,systems=ss)
            push!(pages,page)
            ss = []
            p+=1
        end
    end
    system = (id=s,p=p,bars=bs)
    push!(systems,system)
    push!(ss,s)
    page = (id=p,systems=ss)
    push!(pages,page)

    return notes, chords, bars, systems, pages

end


abstract type Id <: CHARM.Id end
abstract type Constituent <: CHARM.Constituent end
abstract type Hierarchy <: CHARM.Hierarchy end

struct Note{n} <: Id end
struct Chord{n} <: Id end
struct Bar{n} <: Id end
struct System{n} <: Id end
struct Page{n} <: Id end

struct Con{T} <: Constituent end

struct File <: Hierarchy 
    notes
    chords
    bars
    systems
    pages
    File() = new([],[],[],[],[])
    File(fn) = new(parse(fn)...)
end 

abstract type Attribute{N,T} <: CHARM.Attribute{N,T} end

struct FRET <: Attribute{:FRET,String} end
CHARM.__attributes__(::Val{:FRET}) = FRET()

struct COURSE <: Attribute{:COURSE,Int} end
CHARM.__attributes__(::Val{:COURSE}) = COURSE()

struct BAR <: Attribute{:BAR,String} end
CHARM.__attributes__(::Val{:BAR}) = BAR()

CHARM.fnd(x::Note{n},f::File) where n = n > length(f.notes) ? none : Con{Note{n}}()
CHARM.fnd(x::Chord{n},f::File) where n = n > length(f.chords) ? none : Con{Chord{n}}()
CHARM.fnd(x::Bar{n},f::File) where n = n > length(f.bars) ? none : Con{Bar{n}}()
CHARM.fnd(x::System{n},f::File) where n = n > length(f.systems) ? none : Con{System{n}}()
CHARM.fnd(x::Page{n},f::File) where n = n > length(f.pages) ? none : Con{Page{n}}()

CHARM.pts(::Con{Note{n}}) where n = Id[]
CHARM.pts(::Con{Chord{n}}) where n = Id[Note{x}() for x in chords[n].notes]
CHARM.pts(::Con{Bar{n}}) where n = Id[Chord{x}() for x in bars[n].chords]
CHARM.pts(::Con{System{n}}) where n = Id[Bar{x}() for x in systems[n].bars]
CHARM.pts(::Con{Page{n}}) where n = Id[System{x}() for x in pages[n].systems]

CHARM.dom(f::File) = vcat(Id[Page{p}() for p in 1:length(f.pages)],Id[System{s}() for s in 1:length(f.systems)],Id[Bar{b}() for b in 1:length(f.bars)],Id[Chord{c}() for c in 1:length(f.chords)],Id[Note{n}() for n in 1:length(f.notes)])

CHARM.getp(::TYPE,::Con{Note{n}}) where n = "Note"
CHARM.getp(::TYPE,::Con{Chord{n}}) where n = "Chord"
CHARM.getp(::TYPE,::Con{Bar{n}}) where n = "Bar"
CHARM.getp(::TYPE,::Con{System{n}}) where n = "System"
CHARM.getp(::TYPE,::Con{Page{n}}) where n = "Page"

CHARM.geta(::FRET,::Con{Note{n}}) where n = notes[n].fret
CHARM.geta(::COURSE,::Con{Note{n}}) where n = notes[n].course
CHARM.geta(::DUR,::Con{Note{n}}) where n = notes[n].dur
CHARM.geta(::DUR,::Con{Chord{n}}) where n = chords[n].dur

CHARM.geta(::BAR,::Con{Note{n}}) where n = notes[n].b
CHARM.geta(::BAR,::Con{Chord{n}}) where n = chords[n].b

end

using Main.Tabcode

## Midi note number interface

In [14]:
module RenaissancePitch

using Main.CHARM
using Main.Tabcode

course = [67,62,57,53,48,42]
frets = Dict("a"=>2,"b"=>4,"c"=>5,"d"=>7,"e"=>9,"f"=>10,"h"=>11)

getmidinote(cs,fs,c,f) = cs[c]+fs[f]

CHARM.geta(::PITCH,x::Tabcode.Con{Tabcode.Note{n}}) where n = begin 
    c = geta(:COURSE,x)
    f = geta(:FRET,x)
    return getmidinote(courses,frets,c,f)
end

end

Main.RenaissancePitch

## Example Representation

In [16]:
module Example

using Main.CHARM, Main.Tabcode, Main.RenaissancePitch

__data__ = Tabcode.File("output.json")

end

Main.Example

## Representing the topline

In [18]:
module Topline

using Main.CHARM, Main.Tabcode, Main.Example

topline = [Tabcode.Note{c.notes[1]}() for c in Example.__data__.chords]

export Topline

struct Id <: CHARM.Id end
struct Con{T} <: CHARM.Constituent 
    Con() = new{Topline}()
end
struct Hierarchy <: CHARM.Hierarchy end

__data__ = Hierarchy()

CHARM.fnd(::Id,::Hierarchy) = Con()
CHARM.pts(::Con{Id}) = topline

CHARM.dom(::Hierarchy) = CHARM.Id[Id()]

end

Main.Topline

## Integrating Hierarchies

In [20]:
module KB

using Main.CHARM
using Main.Tabcode
using Main.Example
using Main.Topline

struct Data <: CHARM.Hierarchy end
__data__ = Data()

CHARM.fnd(x::Tabcode.Id,::Data) = fnd(x,Example)
CHARM.fnd(x::Topline.Id,::Data) = fnd(x,Topline)
CHARM.dom(::Data) = [dom(Topline)...,dom(Example)...]

end

using Main.KB

In [21]:
cs = cts(KB);

In [22]:
fnd(Topline.Id(),KB) |> pts 

Main.CHARM.Id[]

In [23]:
notes = CHARM.sequence(pts(fnd(Topline(),KB)),KB)
unique([geta(:DUR,n) for n in notes])

LoadError: MethodError: objects of type Module are not callable

In [24]:
ds = unique([geta(:DUR,n) for n in notes])

LoadError: UndefVarError: `notes` not defined

In [25]:
length(ds[1])

LoadError: UndefVarError: `ds` not defined

In [26]:
ds[1][1]

LoadError: UndefVarError: `ds` not defined

In [27]:
durnos = Dict('B'=>8.0,'W'=>4.0,'H'=>2.0,'Q'=>1.0,'E'=>0.5,'S'=>0.25,'T'=>0.125,'Y'=>0.0625,'Z'=>0.03125)

getdurno(s) = begin
    x = durnos[s[1]]
    length(s)==1 && return x
    return x + (x/2)
end

struct NDUR <: CHARM.Attribute{:NDUR,Float64} end
CHARM.__attributes__(::Val{:NDUR}) = NDUR()

CHARM.geta(::NDUR,x::Con{Note{n}}) where n = getdurno(geta(:DUR,x))

LoadError: UndefVarError: `Con` not defined

In [28]:
geta(:NDUR,Con(Note{1}()))

LoadError: UndefVarError: `Note` not defined